In [4]:
import tweepy
import pandas as pd
import webbrowser
import csv
import emojis
import matplotlib.pyplot as plt
import datetime as dt
import time

In [5]:
cosumer_key = 'ChHwIHtyLFnRTlGQIvhBtyEnb'
cosumer_secret = 'sMxZbzgfwYPP7zOu8N4mckpCubUKKFvJRXPgheGI3WbFWHjhb7'
auth = tweepy.OAuthHandler(cosumer_key,cosumer_secret)

In [6]:
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)


True

In [7]:
user_pint_input=input("Please insert pin value:")
auth.get_access_token(user_pint_input)


Please insert pin value: 1067523


('1159401461618491392-KOEwOHmoJxZo76YH6sKKXN0mpOd8oc',
 'y7nLatwuAaOC8AEVvr2NgxigTRw38vHQb8dzg7sGpsg37')

In [8]:
api = tweepy.API(auth, wait_on_rate_limit=True)
me = api.me()


In [14]:
with open('recent_tweet.csv','w',encoding="utf-8") as csv_file:
             #making csv file and headers
             c = ['status_id','text','lang','name','time','media_url','hashtag','favorite_count','retweets_count','source']
             w = csv.DictWriter(csv_file,fieldnames=c)
             w.writeheader()
            
             csv_file.close()


In [6]:
def add_recent_tweet_to_csv(tweet_id):
    #add tweet to csv file via tweet id
    status = api.get_status(tweet_id)
    data = []
    c = ['status_id','text','lang','name','time','media_url','hashtag','favorite_count','retweets_count','source']
    
    try :
        media = status.entities['media'][0]['media_url']
    except:
        media = 'None'

    hashtag = ''
    li=['#PSGBarça','#PSGbarca']
    for i in status.entities['hashtags']:
        if i['text'] not in li:
            hashtag += '#'
            hashtag += i['text']
            print(hashtag)
    
    start_date_time = dt.datetime(2021,3,20,0,0)#date of the match
    created_at = ''
    if status.created_at < start_date_time :
        created_at = 'befor game'
    elif start_date_time < status.created_at < start_date_time + dt.timedelta( hours=21 , minutes=45 ):
        created_at = 'during game'
    else :
        created_at = 'after game'
        
    tmp=[]
    tmp.append(status.id)
    tmp.append(status.text)
    tmp.append(status.lang)
    tmp.append(status.user.screen_name)
    tmp.append(created_at)
    tmp.append(media)
    tmp.append(hashtag)
    tmp.append(status.favorite_count)
    tmp.append(status.retweet_count)
    tmp.append(status.source)
    
    with open('recent_tweet.csv','a',newline='',encoding="utf-8") as csv_file:
              w = csv.writer(csv_file)
              w.writerow(tmp)

In [11]:
def make_dict(x):
    tmp = list(x)
    s = sum(list(x.values()))
    c = 0

    for i in range(len(tmp)):
        
        if i>7 or x[tmp[i]]<s * 0.08:
            c+=x[tmp[i]]
            del x[tmp[i]]
    
    x['other'] = c
    return sort_dict(x)

In [7]:
def write_in_file(x):
    with open('max.txt','w') as file:
        file.write(str(x))
        file.close()

In [8]:
def read_from_file():
    with open('max.txt','r') as file:
        x = file.read()
        file.close()
        return int(x)

In [9]:
def search():
    maximum = read_from_file()
    search = api.search('#psgbarca',count=100)
    m = 0
    for status in search:
        if status.id > maximum:
            try:
                if status.id > m:
                    m = status.id
                add_recent_tweet_to_csv(status.id)
                print(status.id)
            except :
                print('erorr!')
    
    write_in_file(m)

In [ ]:
while(1):
    search()
    time.sleep(5*60)
    #it requests every 5 minutes